In [ ]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_full = len(dm.train_dataloader())
l_val_full = len(dm.val_dataloader())
l_test_full = len(dm.test_dataloader())
print(l_train_full, l_val_full, l_test_full)


In [ ]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/fast_config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_fast = len(dm.train_dataloader())
l_val_fast = len(dm.val_dataloader())
l_test_fast = len(dm.test_dataloader())
print(l_train_fast, l_val_fast, l_test_fast)

In [ ]:
print(l_train_fast/l_train_full)
print(l_val_fast/l_val_full)
print(l_test_fast/l_test_full)

In [ ]:
dummpy = torch.randn(8, 52, 2304)
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/experiments/pretrained_ae_dlinear_indc_indp/config.yaml").dlinear
model = DLinear(cfg)
model2 = DLinear2(cfg)
model.train()
model2.train()
state_dict = model.state_dict()
model2.load_state_dict(state_dict)
out1 = model(dummpy)
out2 = model2(dummpy)
#check all parameters are same
flag = True
for p1, p2 in zip(model.parameters(), model2.parameters()):
    if not torch.allclose(p1, p2, atol=1e-4):
        flag = False
        break
print("Parameters match:", flag)
print(out1.shape, out2.shape)
print(torch.allclose(out1, out2, atol=1e-12))  #
target = torch.randn(8, 48, 2304) 
loss1 = F.mse_loss(out1, target)
loss2 = F.mse_loss(out2, target)

loss1.backward()
loss2.backward()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)
optim2 = torch.optim.SGD(model2.parameters(), lr=1e-3)
optim.step()
optim2.step()

# Check if the gradients are the same
flag = True
for (name1, p1), (name2, p2) in zip(model.named_parameters(), model2.named_parameters()):
    if p1.grad is None or p2.grad is None:
        print(f"Gradient is None for: {name1}, {name2}")
        assert p1.grad is None and p2.grad is None, "both gradients should be None"
        print("skipping gradient check for None gradients")
        continue
    if not torch.allclose(p1.grad, p2.grad, atol=1e-4):
        flag = False
        break
print("Gradients match:", flag)

out1_again = model(dummpy)
out2_again = model2(dummpy)
print(torch.allclose(out1_again, out2_again, atol=1e-12))  # should be True after training

In [ ]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.act1 = nn.GELU()
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
            
        self.act2 = nn.GELU()

    def forward(self, x):
        shortcut = self.shortcut(x)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += shortcut
        return self.act2(out)

class UpsampleBlock(nn.Module):
    def __init__(self, in_ch, out_ch, scale_factor):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='nearest')
        self.resblock = ResidualBlock(in_ch, out_ch, stride=1)

    def forward(self, x):
        return self.resblock(self.upsample(x))

class ConvAutoencoder(nn.Module):
    def __init__(self, in_ch=1, latent_dim=512):
        super().__init__()
        
        # Encoder: 128 → 64 → 32 → 16 → 4 → 1
        self.enc1 = ResidualBlock(in_ch,   64, stride=2)  # 128 → 64
        self.enc2 = ResidualBlock(64,     128, stride=2)  # 64 → 32
        self.enc3 = ResidualBlock(128,    256, stride=2)  # 32 → 16
        self.enc4 = ResidualBlock(256,    512, stride=4)  # 16 → 4
        self.enc5 = ResidualBlock(512,   1024, stride=4)  # 4 → 1

        self.flatten = nn.Flatten()                     # (B,1024,1,1) → (B,1024)
        self.fc_enc = nn.Linear(1024, latent_dim)

        self.fc_dec = nn.Linear(latent_dim, 1024)
        self.unflatten = nn.Unflatten(1, (1024, 1, 1))   # (B,1024) → (B,1024,1,1)
        self.dec_init_conv = ResidualBlock(1024, 1024, stride=1)

        # Decoder: 1 → 4 → 16 → 32 → 64 → 128
        self.dec1 = UpsampleBlock(1024, 512, scale_factor=4)  # 1 → 4
        self.dec2 = UpsampleBlock(512,  256, scale_factor=4)  # 4 → 16
        self.dec3 = UpsampleBlock(256,  128, scale_factor=2)  # 16 → 32
        self.dec4 = UpsampleBlock(128,   64, scale_factor=2)  # 32 → 64
        
        self.final_upsample = nn.Upsample(scale_factor=2, mode='nearest') # 64 → 128
        self.final_conv = nn.Conv2d(64, in_ch, kernel_size=3, stride=1, padding=1)

    def encode(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)
        x = self.flatten(x)
        z = self.fc_enc(x)
        return z

    def decode(self, z):
        x = self.fc_dec(z)
        x = self.unflatten(x)
        x = self.dec_init_conv(x)
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        x = self.dec4(x)
        x = self.final_upsample(x)
        x = self.final_conv(x)
        return x

    def forward(self, x):
        z = self.encode(x)
        reconstruction = self.decode(z)
        return reconstruction, z

if __name__ == "__main__":
    import torch
    model = ConvAutoencoder()
    x = torch.randn(8, 1, 128, 128)  # Batch of 8 images with 1 channel, 128x128 size
    recon, z = model(x)
    print("Input shape:", x.shape)
    print("Reconstruction shape:", recon.shape)
    print("Latent vector shape:", z.shape)
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params_in_million = params / 1e6
    print(f"Total trainable parameters: {params_in_million:.2f} million")

In [ ]:
path = "/home/vatsal/NWM/sevir_lr/data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5"
import h5py
data = h5py.File(path, 'r')
vil = data['vil'][:]
print("Shape of VIL data:", vil.shape)

In [ ]:
import datetime
import os
os.environ['WANDB_API_KEY'] = '041eda3850f131617ee1d1c9714e6230c6ac4772'
datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
from pytorch_lightning.loggers import WandbLogger
exp_name = "test_run_xyz"
save_dir = os.path.join("experiments", "test", exp_name + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S"))
version = "v1.0"
logger = WandbLogger(project="delete_it", name="test_run", save_dir=save_dir, version=version)

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=1, seq_len=1, stride=1, layout='NTHW')
data.setup()
data.prepare_data()
train_loader = data.train_dataloader()
sample1 = next(iter(train_loader))

In [ ]:
len(train_loader)

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data2 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=8, batch_size=8, seq_len=12, stride=20, layout='NTHW')
data2.setup()
data2.prepare_data()
train_loader2 = data2.train_dataloader()
sample2 = next(iter(train_loader2))

In [ ]:
data2_dict = {}
print(len(train_loader2))
from tqdm import tqdm
for epoch in range(2):
    print(f"Epoch {epoch+1}")
    if epoch == 0:
        for idx, sample in tqdm(enumerate(train_loader2)):
            data = sample['vil']
            data2_dict[data] = idx
    else:
        for sample in tqdm(train_loader2):
            data = sample['vil']
            idx = -1
            for i, d in enumerate(data2_dict.keys()):
                if torch.allclose(d, data, atol=1e-12):
                    idx = i
                    break
            print(f"Found data at index: {idx}")

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data3 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=8, seq_len=1, stride=1, layout='NTHW')
data3.setup()
data3.prepare_data()
train_loader3 = data2.train_dataloader()

In [ ]:
for sample2, sample3 in zip(train_loader2, train_loader3):
    data2 = sample2['vil']
    data3 = sample3['vil']
    if not torch.allclose(data2, data3, atol=1e-12):
        print("Data mismatch found!")
        break

In [ ]:
sample3 = next(iter(train_loader3))
import torch
torch.allclose(sample2['vil'], sample3['vil'], atol=1e-12) 

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader2:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    for bdata in data:
        datas.append(bdata)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    for bdata in data:
        idx = -1
        for i, d in enumerate(datas):
            if torch.allclose(bdata, d, atol=1e-12):
                idx = i
                break
        if idx != -1:
            print(f"Found matching data at index {idx}")
        else:
            print("No matching data found _______________________________-")
            break

In [1]:
import os
import shutil
import time

def get_free_disk_space_gb(path):
    """Returns free disk space in GB for the given path."""
    stat = shutil.disk_usage(path)
    return stat.free / (1024 ** 3)

def cleanup_checkpoints_and_wandb(root_dir):
    total_freed = 0
    for dirpath, _, filenames in os.walk(root_dir):
        # Delete .wandb files
        for fname in filenames:
            if fname.endswith(".wandb"):
                fpath = os.path.join(dirpath, fname)
                try:
                    size_mb = os.path.getsize(fpath) / (1024 * 1024)
                    os.remove(fpath)
                    total_freed += size_mb
                    print(f"Deleted .wandb: {fpath} ({size_mb:.2f} MB)")
                except Exception as e:
                    print(f"Failed to delete {fpath}: {e}")
        # Delete .ckpt files if more than 2 in a directory
        ckpt_files = [f for f in filenames if f.endswith(".ckpt")]
        if len(ckpt_files) > 2:
            # Sort by modification time, keep the 2 newest
            ckpt_paths = [os.path.join(dirpath, f) for f in ckpt_files]
            ckpt_paths.sort(key=lambda x: os.path.getmtime(x), reverse=True)
            for fpath in ckpt_paths[2:]:
                try:
                    size_mb = os.path.getsize(fpath) / (1024 * 1024)
                    os.remove(fpath)
                    total_freed += size_mb
                    print(f"Deleted .ckpt: {fpath} ({size_mb:.2f} MB)")
                except Exception as e:
                    print(f"Failed to delete {fpath}: {e}")
    print(f"Total space freed: {total_freed:.2f} MB")

root = "/home/vatsal/NWM/weatherforecasting"
while True:
    free_gb = get_free_disk_space_gb(root)
    print(f"Free disk space: {free_gb:.2f} GB")
    if free_gb < 3:
        cleanup_checkpoints_and_wandb(root)
    else:
        print("Disk space is sufficient, no cleanup needed.")
    time.sleep(200)


Free disk space: 0.00 GB
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2_2/outputs/ae_v2_16x16x16_lin_disc/wandb/run-20250729_233021-lrp42ayk/run-lrp42ayk.wandb (1.27 MB)
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2_2/outputs/ae_v2_16x16x16_lin_disc/wandb/run-20250730_050408-o9kuedtz/run-o9kuedtz.wandb (407.94 MB)
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2_2/outputs/ae_v2_16x16x16_lin_disc/wandb/run-20250729_232856-eupzfe50/run-eupzfe50.wandb (0.84 MB)
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2_2/outputs/ae_v2_16x16x16_lin_disc/wandb/run-20250729_233150-o9kuedtz/run-o9kuedtz.wandb (788.28 MB)


Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2/outputs/ae_v2_64x8x8_lin_disc/wandb/run-20250729_232128-7qamm6uz/run-7qamm6uz.wandb (2178.28 MB)
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2/outputs/ae_v2_64x8x8_lin/wandb/run-20250729_231544-hun23jye/run-hun23jye.wandb (0.10 MB)
Deleted .wandb: /home/vatsal/NWM/weatherforecasting/experiments/ae_v2/outputs/ae_v2_64x8x8_lin/wandb/run-20250729_231642-hun23jye/run-hun23jye.wandb (0.10 MB)
Total space freed: 3376.81 MB


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ResidualConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, s=1, p=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)
        self.norm1 = nn.GroupNorm(min(32, out_ch), out_ch)
        self.act1 = nn.GELU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, k, 1, p, bias=False)
        self.norm2 = nn.GroupNorm(min(32, out_ch), out_ch)
        self.act2 = nn.GELU()
        self.shortcut = (
            nn.Conv2d(in_ch, out_ch, 1, s, bias=False)
            if in_ch != out_ch or s != 1
            else nn.Identity()
        )

    def forward(self, x):
        res = self.shortcut(x)
        out = self.act1(self.norm1(self.conv1(x)))
        out = self.norm2(self.conv2(out))
        return self.act2(out + res)

class AttentionBlock(nn.Module):
    def __init__(self, channels, num_heads=8):
        super().__init__()
        assert channels % num_heads == 0
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.q = nn.Conv2d(channels, channels, 1)
        self.k = nn.Conv2d(channels, channels, 1)
        self.v = nn.Conv2d(channels, channels, 1)
        self.proj = nn.Conv2d(channels, channels, 1)
        self.num_heads = num_heads
        self.head_dim = channels // num_heads

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        q = self.q(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,3,2)
        k = self.k(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,2,3)
        v = self.v(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,3,2)
        att = torch.matmul(q, k) / math.sqrt(self.head_dim)
        att = F.softmax(att, dim=-1)
        out = torch.matmul(att, v)
        out = out.permute(0,1,3,2).reshape(B, C, H, W)
        return x + self.proj(out)

class ResidualUpBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=4, s=2, p=1):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=k, stride=s, padding=p, bias=False)
        self.res = ResidualConvBlock(out_ch, out_ch)

    def forward(self, x):
        return self.res(self.up(x))

class EncoderBridge4x4(nn.Module):
    def __init__(self, in_ch, latent_dim):
        super().__init__()
        self.attn = AttentionBlock(in_ch)
        self.conv = nn.Conv2d(in_ch, latent_dim, kernel_size=4, bias=False)
        self.act  = nn.GELU()

    def forward(self, x):
        x = self.attn(x)
        x = self.conv(x)
        return self.act(x)

class DecoderBridge4x4(nn.Module):
    def __init__(self, latent_dim, out_ch):
        super().__init__()
        self.conv = nn.ConvTranspose2d(latent_dim, out_ch, kernel_size=4, stride=4, bias=False)
        self.attn = AttentionBlock(out_ch)

    def forward(self, x):
        x = self.conv(x)
        return self.attn(x)

class PowerfulAutoencoder128_4x4(nn.Module):
    def __init__(self, in_ch=1, latent_dim=2048,
                 channels_mult=(1,2,4,8,8), base_ch=64):
        super().__init__()
        # Encoder: 128→4
        self.enc0 = ResidualConvBlock(in_ch, base_ch)
        self.enc1 = ResidualConvBlock(base_ch, base_ch*channels_mult[0], s=2)
        self.enc2 = ResidualConvBlock(base_ch*channels_mult[0], base_ch*channels_mult[1], s=2)
        self.enc3 = ResidualConvBlock(base_ch*channels_mult[1], base_ch*channels_mult[2], s=2)
        self.enc4 = ResidualConvBlock(base_ch*channels_mult[2], base_ch*channels_mult[3], s=2)
        self.enc5 = ResidualConvBlock(base_ch*channels_mult[3], base_ch*channels_mult[4], s=2)
        ch = base_ch * channels_mult[-1]

        self.attn           = AttentionBlock(ch)
        self.encoder_bridge = EncoderBridge4x4(in_ch=ch, latent_dim=latent_dim)
        self.decoder_bridge = DecoderBridge4x4(latent_dim=latent_dim, out_ch=ch)

        # Decoder: 4→128 with ResidualUpBlock
        self.up0 = ResidualUpBlock(ch, base_ch*channels_mult[3])  # 4→8
        self.up1 = ResidualUpBlock(base_ch*channels_mult[3], base_ch*channels_mult[2])  # 8→16
        self.up2 = ResidualUpBlock(base_ch*channels_mult[2], base_ch*channels_mult[1])  # 16→32
        self.up3 = ResidualUpBlock(base_ch*channels_mult[1], base_ch*channels_mult[0])  # 32→64
        self.up4 = ResidualUpBlock(base_ch*channels_mult[0], base_ch)                   # 64→128
        self.final_conv = nn.Conv2d(base_ch, in_ch, 3, 1, 1)

    def encoder(self, x):
        x = self.enc0(x)
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)
        # x = self.attn(x)
        # x = self.encoder_bridge(x)
        # x = torch.flatten(x, 1)
        return x

    def decode(self, z):
        # x = z.reshape(-1, z.size(1), 1, 1)
        x = z
        # x = self.decoder_bridge(x)
        x = self.up0(x)
        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        x = self.final_conv(x)
        return x

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decode(z)
        return recon, z

model = PowerfulAutoencoder128_4x4(in_ch=1, latent_dim=1024, base_ch = 16)
x = torch.randn(8, 1, 128, 128)  # Batch of 8 images with 1 channel, 128x128 size
recon, z = model(x)
print("Input shape:", x.shape)
print("Latent vector shape:", z.shape)
print("Reconstruction shape:", recon.shape)
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
params_in_million = params / 1e6
print(f"Total trainable parameters: {params_in_million:.2f} million")   

Input shape: torch.Size([8, 1, 128, 128])
Latent vector shape: torch.Size([8, 128, 4, 4])
Reconstruction shape: torch.Size([8, 1, 128, 128])
Total trainable parameters: 5.85 million


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ResidualConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, s=1, p=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, k, s, p, bias=False)
        self.norm1 = nn.GroupNorm(min(32, out_ch), out_ch)
        self.act1 = nn.GELU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, k, 1, p, bias=False)
        self.norm2 = nn.GroupNorm(min(32, out_ch), out_ch)
        self.act2 = nn.GELU()
        self.shortcut = (
            nn.Conv2d(in_ch, out_ch, 1, s, bias=False)
            if in_ch != out_ch or s != 1 else nn.Identity()
        )
    def forward(self, x):
        res = self.shortcut(x)
        out = self.act1(self.norm1(self.conv1(x)))
        out = self.norm2(self.conv2(out))
        return self.act2(out + res)

class AttentionBlock(nn.Module):
    def __init__(self, channels, num_heads=8):
        super().__init__()
        assert channels % num_heads == 0
        self.norm = nn.GroupNorm(min(32, channels), channels)
        self.q = nn.Conv2d(channels, channels, 1)
        self.k = nn.Conv2d(channels, channels, 1)
        self.v = nn.Conv2d(channels, channels, 1)
        self.proj = nn.Conv2d(channels, channels, 1)
        self.num_heads = num_heads
        self.head_dim = channels // num_heads
    def forward(self, x):
        B,C,H,W = x.shape
        h = self.norm(x)
        q = self.q(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,3,2)
        k = self.k(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,2,3)
        v = self.v(h).reshape(B, self.num_heads, self.head_dim, H*W).permute(0,1,3,2)
        att = torch.matmul(q, k) / math.sqrt(self.head_dim)
        att = F.softmax(att, dim=-1)
        out = torch.matmul(att, v).permute(0,1,3,2).reshape(B,C,H,W)
        return x + self.proj(out)

class ResidualUpBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k=4, s=2, p=1):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, k, s, p, bias=False)
        self.res = ResidualConvBlock(out_ch, out_ch)
    def forward(self, x):
        return self.res(self.up(x))

# Variant 1: z shape = (B,1024,1,1), downsample via ResidualConvBlock to 4x4
class AutoencoderZ1024(nn.Module):
    def __init__(self, in_ch=1, base_ch=64, latent_ch=1024):
        super().__init__()
        self.enc = nn.Sequential(
            ResidualConvBlock(in_ch, base_ch),          # 128→128
            ResidualConvBlock(base_ch, base_ch, s=2),    # 128→64
            ResidualConvBlock(base_ch, base_ch*2, s=2),  # 64→32
            ResidualConvBlock(base_ch*2, base_ch*4, s=2),# 32→16
            ResidualConvBlock(base_ch*4, base_ch*8, s=2),# 16→8
            ResidualConvBlock(base_ch*8, base_ch*8, s=2) # 8→4
        )
        self.attn = AttentionBlock(base_ch*8)
        self.to_latent   = nn.Conv2d(base_ch*8, latent_ch, 4, bias=False)          # 4→1
        self.from_latent = nn.ConvTranspose2d(latent_ch, base_ch*8, 4, stride=4, bias=False) # 1→4
        self.dec = nn.Sequential(
            ResidualUpBlock(base_ch*8, base_ch*8),  # 4→8
            ResidualUpBlock(base_ch*8, base_ch*4),  # 8→16
            ResidualUpBlock(base_ch*4, base_ch*2),  # 16→32
            ResidualUpBlock(base_ch*2, base_ch),    # 32→64
            ResidualUpBlock(base_ch, base_ch),      # 64→128
            nn.Conv2d(base_ch, in_ch, 3,1,1)
        )
    def forward(self, x):
        x = self.enc(x)
        x = self.attn(x)
        z = self.to_latent(x)     # (B,latent,1,1)
        x = self.from_latent(z)
        x = self.attn(x)
        recon = self.dec(x)
        return recon, z

# Variant 2: z shape = (B,64,4,4), downsample to 4x4 via ResidualConvBlock
class AutoencoderZ64(nn.Module):
    def __init__(self, in_ch=1, base_ch=64, latent_ch=64):
        super().__init__()
        self.enc = nn.Sequential(
            ResidualConvBlock(in_ch, base_ch),        # 64→64
            ResidualConvBlock(base_ch, base_ch, s = 2),        # 64→64
            ResidualConvBlock(base_ch, base_ch*2, s=2),  # 64→32
            ResidualConvBlock(base_ch*2, base_ch*4, s=2),# 32→16
            ResidualConvBlock(base_ch*4, base_ch*8, s=2),# 16→8
            ResidualConvBlock(base_ch*8, base_ch*16, s=2) # 8→4
        )
        self.attn = AttentionBlock(base_ch*16)
        self.to_latent   = ResidualConvBlock(base_ch*16, latent_ch, k=3, s=1, p=1)
        self.from_latent = ResidualConvBlock(latent_ch, base_ch*16, k=3, s=1, p=1)
        self.dec = nn.Sequential(
            AttentionBlock(base_ch*16),
            ResidualUpBlock(base_ch*16, base_ch*8), # 4→8
            ResidualUpBlock(base_ch*8, base_ch*4), # 8→16
            ResidualUpBlock(base_ch*4, base_ch*2),   # 16→32
            ResidualUpBlock(base_ch*2, base_ch),     # 32→64
            ResidualUpBlock(base_ch, base_ch),
            nn.Conv2d(base_ch, in_ch, 3,1,1)
        )
    def forward(self, x):
        x = self.enc(x)
        x = self.attn(x)
        z = self.to_latent(x)    # (B,latent,4,4)
        x = self.from_latent(z)
        x = self.attn(x)
        recon = self.dec(x)
        return recon, z

class AutoencoderZ16(nn.Module):
    def __init__(self, in_ch=1, base_ch=128, latent_ch=16):
        super().__init__()
        self.enc = nn.Sequential(
            ResidualConvBlock(in_ch, base_ch),        # 64→64
            ResidualConvBlock(base_ch, base_ch, s = 2),
            ResidualConvBlock(base_ch, base_ch*2, s=2),  # 64→32
            ResidualConvBlock(base_ch*2, base_ch*4, s=2),# 32→16
            ResidualConvBlock(base_ch*4, base_ch*8, s=2),# 16→8
        )
        self.attn = AttentionBlock(base_ch*8)
        self.to_latent   = ResidualConvBlock(base_ch*8, latent_ch, k=3, s=1, p=1)
        self.from_latent = ResidualConvBlock(latent_ch, base_ch*8, k=3, s=1, p=1)
        self.dec = nn.Sequential(
            AttentionBlock(base_ch*8),
            ResidualUpBlock(base_ch*8, base_ch*4), # 8→16
            ResidualUpBlock(base_ch*4, base_ch*2),   # 16→32
            ResidualUpBlock(base_ch*2, base_ch),     # 32→64
            ResidualUpBlock(base_ch, base_ch),
            nn.Conv2d(base_ch, in_ch, 3,1,1)
        )
    def forward(self, x):
        x = self.enc(x)
        x = self.attn(x)
        z = self.to_latent(x)    # (B,latent,4,4)
        x = self.from_latent(z)
        x = self.attn(x)
        recon = self.dec(x)
        return recon, z
    
models = [AutoencoderZ1024(), AutoencoderZ64(), AutoencoderZ16()]
for model in models:
    x = torch.randn(8, 1, 128, 128)  # Batch of 8 images with 1 channel, 128x128 size
    recon, z = model(x)
    print("Input shape:", x.shape)
    print("Latent vector shape:", z.shape)
    print("Reconstruction shape:", recon.shape)
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params_in_million = params / 1e6
    print(f"Total trainable parameters: {params_in_million:.2f} million")

Input shape: torch.Size([8, 1, 128, 128])
Latent vector shape: torch.Size([8, 1024, 1, 1])
Reconstruction shape: torch.Size([8, 1, 128, 128])
Total trainable parameters: 41.11 million
Input shape: torch.Size([8, 1, 128, 128])
Latent vector shape: torch.Size([8, 64, 4, 4])
Reconstruction shape: torch.Size([8, 1, 128, 128])
Total trainable parameters: 56.36 million
Input shape: torch.Size([8, 1, 128, 128])
Latent vector shape: torch.Size([8, 16, 8, 8])
Reconstruction shape: torch.Size([8, 1, 128, 128])
Total trainable parameters: 55.67 million


In [21]:
import os

def get_ckpt_total_size(root_dir):
    total_size = 0
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith('.ckpt'):
                fpath = os.path.join(dirpath, fname)
                total_size += os.path.getsize(fpath)
    return total_size

def format_size(bytes_size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024
    return f"{bytes_size:.2f} PB"

root = "/home/vatsal/NWM"
total = get_ckpt_total_size(root)
print("Total .ckpt size:", format_size(total))


Total .ckpt size: 5.23 GB


In [31]:
import os

def get_dir_size(path):
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fpath = os.path.join(dirpath, f)
            try:
                if not os.path.islink(fpath) and os.path.isfile(fpath):
                    total += os.path.getsize(fpath)
            except:
                continue
    return total

def list_top_dirs(root_dir, top_n=20):
    dir_sizes = []
    for entry in os.scandir(root_dir):
        if entry.is_dir(follow_symlinks=False):
            size = get_dir_size(entry.path)
            dir_sizes.append((entry.path, size))
    
    dir_sizes.sort(key=lambda x: x[1], reverse=True)
    for path, size in dir_sizes[:top_n]:
        print(f"{path}: {size / (1024**3):.2f} GB")

# Example usage
list_top_dirs("/home/vatsal/NWM", top_n=20)


/home/vatsal/NWM/weatherforecasting: 20.60 GB
/home/vatsal/NWM/weather: 11.77 GB
/home/vatsal/NWM/sevir_lr: 8.77 GB
/home/vatsal/NWM/PreDiff: 0.36 GB
/home/vatsal/NWM/DiffCast: 0.16 GB
/home/vatsal/NWM/AlphaPre: 0.00 GB
/home/vatsal/NWM/.vscode: 0.00 GB
/home/vatsal/NWM/wandb: 0.00 GB


In [ ]:
import torch
import torch.nn as nn

class PositionalEncoder(nn.Module):
    """
    Adds x, y coordinate information to the feature map.
    This is a non-learnable layer.
    """
    def __init__(self):
        super(PositionalEncoder, self).__init__()

    def forward(self, x):
        """
        Args:
            x (Tensor): The input feature map of shape (B, C, H, W).
        Returns:
            Tensor: The output tensor with 2 extra channels for x and y coords.
                    Shape will be (B, C+2, H, W).
        """
        # Get the shape of the input tensor
        b, c, h, w = x.shape
        
        # Create coordinate grids
        # y_coords are horizontal lines, x_coords are vertical lines
        y_coords = torch.linspace(-1, 1, h, device=x.device).unsqueeze(1).repeat(1, w)
        x_coords = torch.linspace(-1, 1, w, device=x.device).unsqueeze(0).repeat(h, 1)
        
        # Add a channel dimension and batch dimension
        # Shape becomes (1, 1, H, W)
        y_coords = y_coords.unsqueeze(0).unsqueeze(0)
        x_coords = x_coords.unsqueeze(0).unsqueeze(0)
        
        # Repeat for the entire batch
        # Shape becomes (B, 1, H, W)
        y_coords = y_coords.repeat(b, 1, 1, 1)
        x_coords = x_coords.repeat(b, 1, 1, 1)
        
        # Concatenate along the channel dimension
        # (B, C, H, W) + (B, 1, H, W) + (B, 1, H, W) -> (B, C+2, H, W)
        return torch.cat([x, y_coords, x_coords], dim=1)


class Encoder(nn.Module):
    """
    Encodes a 1x128x128 image down to a 64x8x8 feature map.
    """
    def __init__(self):
        super(Encoder, self).__init__()
        self.model = nn.Sequential(
            # Input: 1 x 128 x 128
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 16 x 64 x 64

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 32 x 32 x 32

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 64 x 16 x 16
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 64 x 8 x 8
        )

    def forward(self, x):
        return self.model(x)


class Decoder(nn.Module):
    """
    Decodes the flattened latent vector back to a 1x128x128 image.
    """
    def __init__(self):
        super(Decoder, self).__init__()
        
        # The first layer processes the combined feature and positional channels
        # It learns to interpret the positional data
        self.initial_conv = nn.Sequential(
            nn.Conv2d(in_channels=66, out_channels=64, kernel_size=1, stride=1, padding=0),
            nn.ReLU(True)
        )
        
        self.upsample_model = nn.Sequential(
            # Input: 64 x 8 x 8
            nn.Upsample(scale_factor=2, mode='nearest'), # -> 64 x 16 x 16
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='nearest'), # -> 64 x 32 x 32
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            
            nn.Upsample(scale_factor=2, mode='nearest'), # -> 32 x 64 x 64
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),

            nn.Upsample(scale_factor=2, mode='nearest'), # -> 16 x 128 x 128
            # Final convolution to get back to 1 channel
            nn.Conv2d(16, 1, kernel_size=3, stride=1, padding=1),
            
            # Apply Sigmoid to scale output pixels between 0 and 1
            nn.Sigmoid()
        )

    def forward(self, z):
        # Reshape the flattened vector back to its 3D form (including pos channels)
        # Original size: 66 * 8 * 8 = 4224
        # The -1 in batch size infers it from the input
        x = z.reshape(-1, 66, 8, 8)
        
        # Process with the initial 1x1 convolution
        x = self.initial_conv(x)
        
        # Pass through the upsampling layers
        reconstructed_image = self.upsample_model(x)
        return reconstructed_image


class Autoencoder(nn.Module):
    """
    The full autoencoder model combining the encoder, positional encoding,
    flattening, and the decoder.
    """
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.pos_encoder = PositionalEncoder()
        self.decoder = Decoder()

    def forward(self, x):
        # 1. Encode the input image
        features = self.encoder(x)
        
        # 2. Add positional embeddings
        features_with_pos = self.pos_encoder(features)
        
        # 3. Flatten the result to create the latent vector 'z'
        # The .reshape() method flattens the tensor while keeping the batch dimension
        z = features_with_pos.reshape(features_with_pos.size(0), -1)
        
        # 4. Decode the latent vector to reconstruct the image
        reconstruction = self.decoder(z)
        
        return reconstruction, z, features, features_with_pos


# --- Main Execution ---
if __name__ == '__main__':
    # Check if a CUDA-enabled GPU is available, otherwise use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize the autoencoder model and move it to the selected device
    model = Autoencoder().to(device)
    
    # Create a dummy input image tensor (batch size = 4)
    # The image is 1 channel (grayscale), 128x128 pixels
    batch_size = 4
    input_image = torch.randn(batch_size, 1, 128, 128).to(device)
    
    print(f"Input image shape: \t\t{input_image.shape}")
    print("-" * 50)
    
    # Pass the image through the model
    reconstructed_image, latent_vector, encoded_features, features_pos = model(input_image)
    
    # Print the shapes at each stage to verify the architecture
    print("--- Forward Pass Shapes ---")
    print(f"1. After Encoder: \t\t{encoded_features.shape}")
    print(f"2. After Positional Encoding: \t{features_pos.shape}")
    print(f"3. Flattened Latent Vector (z): {latent_vector.shape}")
    print(f"4. Reconstructed Image: \t{reconstructed_image.shape}")
    print("-" * 50)
    
    # You can also check the number of parameters
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,}")

Using device: cuda
Input image shape: 		torch.Size([4, 1, 128, 128])
--------------------------------------------------
--- Forward Pass Shapes ---
1. After Encoder: 		torch.Size([4, 64, 8, 8])
2. After Positional Encoding: 	torch.Size([4, 66, 8, 8])
3. Flattened Latent Vector (z): torch.Size([4, 4224])
4. Reconstructed Image: 	torch.Size([4, 1, 128, 128])
--------------------------------------------------
Total trainable parameters: 124,673


In [1]:
import torch, numpy as np, h5py, tqdm
from sklearn.decomposition import PCA
import tqdm
HDF5_PATH   = '/home/vatsal/NWM/sevir_lr/data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5'   
DS_NAME     = 'vil'           
SAMPLE_STEP = 1         
PATCH_SIZE  = 128            
GPU         = torch.cuda.is_available()
# --------------------------------

frames = []
with h5py.File(HDF5_PATH, 'r') as f:
    imgs = f[DS_NAME][:]                     # (T, H, W) or (T, C, H, W)
    if imgs.ndim == 4 and imgs.shape[1] == 1:
        imgs = imgs[:, 0]
    imgs = imgs[::SAMPLE_STEP]
    for t in tqdm.tqdm(imgs, desc='loading'):
        for x in range(0, t.shape[-1]):
            img = t[..., x]
            frames.append(img.flatten())
X = np.stack(frames)        # (N, 16384) for 128²

# 2. Exact PCA on GPU (or CPU)
print('Fitting PCA on', X.shape)
pca = PCA().fit(X)

# 3. Energy curve
cum = np.cumsum(pca.explained_variance_ratio_)
k95 = int(np.searchsorted(cum, 0.95)) + 1
print(f'Need k={k95} PCs for 95 % variance')

# 4. Latent tensor bound
H, W = PATCH_SIZE, PATCH_SIZE
# Empirical rule: latent volume ≥ k95
# Let r = spatial downsample ratio, c = channels
def smallest_latent(k, r_list=[2,4,8]):
    for r in r_list:
        for c in [8,16,32,64]:
            if (H//r)*(W//r)*c >= k:
                return (H//r, W//r, c), (H//r)*(W//r)*c
    return None, None
shape, vol = smallest_latent(k95)
print('Smallest 3-D latent tensor (H/r, W/r, c):', shape, 'volume =', vol)

loading: 100%|██████████| 1440/1440 [00:00<00:00, 2760.24it/s]


Fitting PCA on (36000, 16384)
Need k=1062 PCs for 95 % variance
Smallest 3-D latent tensor (H/r, W/r, c): (64, 64, 8) volume = 32768


In [4]:
H, W = PATCH_SIZE, PATCH_SIZE
# Empirical rule: latent volume ≥ k95
# Let r = spatial downsample ratio, c = channels
def smallest_latent(k, r_list=[2,4,8,16,32,64,128]):
    for r in r_list:
        for c in [8,16,32,64,128,256]:
            if (H//r)*(W//r)*c >= k:
                print(f"H={H//r}, W={W//r}, c={c}")
                # return (H//r, W//r, c), (H//r)*(W//r)*c
    return None, None
shape, vol = smallest_latent(k95)
print('Smallest 3-D latent tensor (H/r, W/r, c):', shape, 'volume =', vol)

H=64, W=64, c=8
H=64, W=64, c=16
H=64, W=64, c=32
H=64, W=64, c=64
H=64, W=64, c=128
H=64, W=64, c=256
H=32, W=32, c=8
H=32, W=32, c=16
H=32, W=32, c=32
H=32, W=32, c=64
H=32, W=32, c=128
H=32, W=32, c=256
H=16, W=16, c=8
H=16, W=16, c=16
H=16, W=16, c=32
H=16, W=16, c=64
H=16, W=16, c=128
H=16, W=16, c=256
H=8, W=8, c=32
H=8, W=8, c=64
H=8, W=8, c=128
H=8, W=8, c=256
H=4, W=4, c=128
H=4, W=4, c=256
Smallest 3-D latent tensor (H/r, W/r, c): None volume = None


output shape: torch.Size([2, 3, 128, 128])
trainable params: 17.2 M


In [23]:
import torch
import torch.nn as nn

# ------------------------------------------------------------
# Pre-Act Bottleneck (no SE, simple, stable)
# channels -> channels, 3×3 grouped conv
# ------------------------------------------------------------
class Bottleneck(nn.Module):
    def __init__(self, channels: int, groups: int = 8):
        super().__init__()
        mid = channels // 4
        # ensure groups divides mid
        g = min(groups, mid)
        assert mid % g == 0, f"groups ({g}) must divide mid channels ({mid})"
        self.f = nn.Sequential(
            nn.BatchNorm2d(channels), nn.GELU(),
            nn.Conv2d(channels, mid, 1, bias=False),
            nn.BatchNorm2d(mid), nn.GELU(),
            nn.Conv2d(mid, mid, 3, padding=1, groups=g, bias=False),
            nn.BatchNorm2d(mid), nn.GELU(),
            nn.Conv2d(mid, channels, 1, bias=False)
        )

    def forward(self, x):
        return x + self.f(x)

# ------------------------------------------------------------
# Encoder block: stride-2 conv + N Bottlenecks
# ------------------------------------------------------------
class EncBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, num_blocks: int = 2, groups: int = 8):
        super().__init__()
        self.down = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch), nn.GELU()
        )
        self.res = nn.Sequential(*[Bottleneck(out_ch, groups=groups) for _ in range(num_blocks)])

    def forward(self, x):
        return self.res(self.down(x))

# ------------------------------------------------------------
# Decoder block: stride-2 transposed conv + N Bottlenecks
# ------------------------------------------------------------
class DecBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, num_blocks: int = 2, groups: int = 8):
        super().__init__()
        self.up = nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch), nn.GELU()
        )
        self.res = nn.Sequential(*[Bottleneck(out_ch, groups=groups) for _ in range(num_blocks)])

    def forward(self, x):
        return self.res(self.up(x))

# ------------------------------------------------------------
# Full AE with optional activation
# ------------------------------------------------------------
class PosAwareAutoEncoder(nn.Module):
    def __init__(
        self,
        in_channels: int = 1,
        latent_channels: int = 16,
        groups: int = 8,
    ):
        super().__init__()
        self.latent_channels = latent_channels

        # ------------- Encoder -------------
        self.enc = nn.Sequential(
            EncBlock(in_channels, 256, num_blocks=4, groups=groups),
            EncBlock(256, 512, num_blocks=4, groups=groups),
            EncBlock(512, 1024, num_blocks=4, groups=groups),
            nn.Conv2d(1024, latent_channels, 1)
        )

        # ------------- Positional embedding -------------
        self.pos_emb = nn.Parameter(torch.randn(latent_channels, 16, 16))

        # ------------- Decoder -------------
        self.dec = nn.Sequential(
            nn.Conv2d(latent_channels, 1024, 1),
            DecBlock(1024, 512, num_blocks=4, groups=groups),
            DecBlock(512, 256, num_blocks=4, groups=groups),
            DecBlock(256, 128, num_blocks=4, groups=groups),
            nn.Conv2d(128, in_channels, 3, padding=1)
        )

        self.act = nn.Sigmoid()

    def encode(self, x):
        z = self.enc(x)
        z = z + self.pos_emb
        return z.flatten(1)

    def decode(self, z_flat):
        z = z_flat.view(-1, self.latent_channels, 16, 16)
        return self.act(self.dec(z))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z), z

# ------------------------------------------------------------
# Quick test
# ------------------------------------------------------------
if __name__ == "__main__":
    net = PosAwareAutoEncoder()
    x = torch.randn(2, 1, 128, 128)
    y, z = net(x)
    print("output:", y.shape)
    print("latent:", z.shape)
    params = sum(p.numel() for p in net.parameters()) / 1e6
    print(f"trainable params: {params:.1f} M")


output: torch.Size([2, 1, 128, 128])
latent: torch.Size([2, 4096])
trainable params: 25.5 M


In [8]:
import torch
import torch.nn as nn
import math

# ------------------------------------------------------------
# CNN building blocks (unchanged)
# ------------------------------------------------------------
class Bottleneck(nn.Module):
    def __init__(self, channels: int, groups: int = 8):
        super().__init__()
        mid = channels // 4
        g = min(groups, mid)
        if g <= 0: g = 1
        while mid % g != 0: g -= 1
        if g <= 0: g = 1
        self.f = nn.Sequential(
            nn.BatchNorm2d(channels), nn.GELU(),
            nn.Conv2d(channels, mid, 1, bias=False),
            nn.BatchNorm2d(mid), nn.GELU(),
            nn.Conv2d(mid, mid, 3, padding=1, groups=g, bias=False),
            nn.BatchNorm2d(mid), nn.GELU(),
            nn.Conv2d(mid, channels, 1, bias=False)
        )
    def forward(self, x): return x + self.f(x)

class EncBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, num_blocks: int = 2, groups: int = 8):
        super().__init__()
        self.down = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch), nn.GELU()
        )
        self.res = nn.Sequential(*[Bottleneck(out_ch, groups=groups) for _ in range(num_blocks)])
    def forward(self, x): return self.res(self.down(x))

class DecBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, num_blocks: int = 2, groups: int = 8):
        super().__init__()
        self.up = nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_ch), nn.GELU()
        )
        self.res = nn.Sequential(*[Bottleneck(out_ch, groups=groups) for _ in range(num_blocks)])
    def forward(self, x): return self.res(self.up(x))

# ------------------------------------------------------------
# The Final, Balanced Hybrid CNN-Transformer Autoencoder
# ------------------------------------------------------------
class HybridAE_8x8_Grid(nn.Module):
    def __init__(
        self,
        in_channels: int = 1,
        img_size: int = 128,
        # We will create a grid of [64, 8, 8] to get our 4096 latent vector
        latent_channels: int = 64,
        # CNN goes deeper to reduce spatial size
        cnn_channels: list = [64, 128, 256, 512],
        cnn_blocks: int = 4,
        # Transformer parameters
        tr_depth: int = 8,
        tr_heads: int = 8,
    ):
        super().__init__()
        self.latent_channels = latent_channels
        self.cnn_final_channels = cnn_channels[-1]

        # Calculate grid size after deeper CNN encoding
        self.grid_size = img_size // (2 ** len(cnn_channels)) # 128 / (2^4) = 8
        self.n_tokens = self.grid_size ** 2 # 8*8 = 64

        # The final 1D latent vector size is now correct by design
        self.latent_dim = self.n_tokens * self.latent_channels # 64 * 64 = 4096
        
        # ------------- 1. Deeper CNN Encoder Head -------------
        enc_layers = []
        ch_in = in_channels
        for ch_out in cnn_channels:
            enc_layers.append(EncBlock(ch_in, ch_out, num_blocks=cnn_blocks))
            ch_in = ch_out
        self.cnn_encoder = nn.Sequential(*enc_layers)
        # Output is now [B, 512, 8, 8]

        # ------------- 2. Projection to Latent Channel Dimension -------------
        self.to_latent_channels = nn.Conv2d(self.cnn_final_channels, self.latent_channels, 1)
        # Output is now [B, 64, 8, 8]

        # ------------- 3. Transformer Encoder Body -------------
        self.pos_embed = nn.Parameter(torch.zeros(1, self.n_tokens, self.latent_channels))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.latent_channels, nhead=tr_heads, dim_feedforward=self.latent_channels * 4,
            dropout=0.1, activation='gelu', batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=tr_depth)

        # ------------- 4. Transformer Decoder Body -------------
        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, self.n_tokens, self.latent_channels))
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=self.latent_channels, nhead=tr_heads, dim_feedforward=self.latent_channels * 4,
            dropout=0.1, activation='gelu', batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=tr_depth)

        # ------------- 5. Projection from Latent Channel Dimension -------------
        self.from_latent_channels = nn.Conv2d(self.latent_channels, self.cnn_final_channels, 1)
        
        # ------------- 6. Deeper CNN Decoder Tail -------------
        dec_layers = []
        ch_in = self.cnn_final_channels
        for ch_out in reversed(cnn_channels[:-1]):
            dec_layers.append(DecBlock(ch_in, ch_out, num_blocks=cnn_blocks))
            ch_in = ch_out
        dec_layers.append(DecBlock(cnn_channels[0], cnn_channels[0], num_blocks=cnn_blocks))
        self.cnn_decoder = nn.Sequential(*dec_layers)

        self.output_conv = nn.Conv2d(cnn_channels[0], in_channels, 3, padding=1)
        self.act = nn.Sigmoid()

    def encode(self, x):
        # 1. CNN downsampling to a small spatial grid
        x = self.cnn_encoder(x) # -> [B, 512, 8, 8]

        # 2. Project to the target latent channel dimension
        x = self.to_latent_channels(x) # -> [B, 64, 8, 8]

        # 3. Reshape for Transformer and add position embedding
        x = x.flatten(2).transpose(1, 2) # -> [B, 64, 64]
        x = x + self.pos_embed

        # 4. Process sequence with Transformer
        x = self.transformer_encoder(x) # -> [B, 64, 64]

        # 5. Flatten to create the final 1D latent vector 'z'
        z = x.flatten(1) # -> [B, 64 * 64] = [B, 4096]
        return z

    def decode(self, z):
        # 1. Reshape 'z' from 1D vector back to a sequence
        x = z.view(-1, self.n_tokens, self.latent_channels) # -> [B, 64, 64]

        # 2. Transformer Decoder
        memory = x
        decoder_input = torch.zeros_like(x, device=z.device)
        decoder_input = decoder_input + self.decoder_pos_embed
        x = self.transformer_decoder(tgt=decoder_input, memory=memory) # -> [B, 64, 64]

        # 3. Reshape back to an image-like grid
        x = x.transpose(1, 2).view(-1, self.latent_channels, self.grid_size, self.grid_size) # -> [B, 64, 8, 8]

        # 4. Project back to the original CNN channel dimension
        x = self.from_latent_channels(x) # -> [B, 512, 8, 8]

        # 5. CNN upsampling
        x = self.cnn_decoder(x) # -> [B, 64, 128, 128]

        x = self.output_conv(x) # -> [B, 1, 128, 128]
        return self.act(x)

    def forward(self, x):
        z = self.encode(x)
        recon = self.decode(z)
        return recon, z

# ------------------------------------------------------------
# Quick test
# ------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    net = HybridAE_8x8_Grid(
        latent_channels = 64,      # unchanged
        # --------------  W I D E R   C N N  -----------------
        cnn_channels = [128, 256, 512, 1024],   # 2× wider
        cnn_blocks = 6,                          # deeper too
        # --------------  T R A N S F O R M E R  -------------
        tr_depth = 12,                           # deeper
        tr_heads = 8,   
    ).to(device)

    x = torch.randn(2, 1, 128, 128).to(device)
    y, z = net(x)

    print(f"\n--- Model Test ---")
    print("Final Latent Dimension (z.shape[1]):", z.shape[1])
    print("Output shape:", y.shape)
    print("Latent shape:", z.shape)

    params = sum(p.numel() for p in net.parameters() if p.requires_grad) / 1e6
    print(f"Total trainable params: {params:.2f} M")

Using device: cuda

--- Model Test ---
Final Latent Dimension (z.shape[1]): 4096
Output shape: torch.Size([2, 1, 128, 128])
Latent shape: torch.Size([2, 4096])
Total trainable params: 29.88 M
